In [1]:
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
import tensorflow as tf
sess = tf.InteractiveSession()

In [5]:
knn__ = 12
batch_size__ = 32

uniforms_ = []
peaks_ = []

for i in range(1):

    if i % 10 == 0:
        print(i)
    
    # initialize exponential mass
    m_np = np.random.exponential(80., [batch_size__,1])+80.
    #m_np = np.array([120.,120.,120.,120.,120.,120.])
    #m_np = np.array([100.,150.,140.,130.,125.,110.])
    m__ = tf.constant(m_np, dtype=tf.float32)

    # init binary class
    #y_true_np = np.random.randint(2, size=batch_size__)
    y_true_np = np.zeros(batch_size__)
    y_true__ = tf.constant(y_true_np, dtype=tf.int32)

    # init preds
    y_pred_np = np.random.rand(batch_size__,1)
    #y_pred_np = np.array([0.6,0.6,0.6,0.6,0.6,0.6])
    #y_pred_np = np.array([0.1,0.2,.9,.8,0.5,0.6])
    # uniform distribution
    y_pred__ = tf.constant(y_pred_np, dtype=tf.float32)
    uniforms_.append(cvm_loss_grad(m__, y_pred__, y_true__, knn__, debug=False)[0].eval())
    print(uniforms_[-1])
    # peaked: artificially set masses near 125 to have y_pred=1.
    y_pred_np[(m_np > 115.) & (m_np < 135.)] = 1.
    y_pred__ = tf.constant(y_pred_np, dtype=tf.float32)
    peaks_.append(cvm_loss_grad(m__, y_pred__, y_true__, knn__, debug=False)[0].eval())
    print(peaks_[-1])

0
0.0272856
0.0647759


In [3]:
def cvm_loss_grad_i(m_i, y_pred, m, K, debug=False):
    '''
    Calculate CVM gradient and loss contribution for the ith sample.
    CVM is expressed in terms of the CDF of the kNN(i) prediction scores s, F_knn(i)_(s)
    and of a reference uniform CDF, F_(s).
    
    Args:
        m_i: scalar representing mass of the ith sample
        y_pred: array of prediction scores (including the ith sample)
        m: array of mass values (including the ith sample)
        K: scalar representing number of nearest neighbors to use (including the ith sample)
    Returns:
        loss_i: contribution of the ith sample to the CVM loss
        grad_i: CVM gradient wrt y_pred_i
    
    '''
    if debug: print('m_i:',m_i.eval())
    
    # Get distances |m_i - m|
    dist = tf.abs(tf.subtract(m_i, tf.squeeze(m)))
    if debug: print('dist',dist.eval())
    
    # Get kNN of m_i:
    # Sort distances by size and get first K elements
    _, indices_knn = tf.nn.top_k(tf.negative(dist), k=K)
    #_, indices_knn = tf.nn.top_k(tf.reshape(tf.negative(dist), [1,-1]), k=K)
    # Then, indices_knn[0] gives position of index i in y_pred
    i = tf.squeeze(indices_knn)[0]
    if debug: print('idx:',i.eval())
    
    # Extract elements in y_pred corresponding to kNN
    y_pred_knn_dist_sorted = tf.gather(tf.squeeze(y_pred), indices_knn) # these are arranged by increasing distance
    # The 0th element corresponds to y_pred_i
    y_pred_i = y_pred_knn_dist_sorted[0]
    if debug: print('y_pred_knn',y_pred_knn_dist_sorted.eval())
    # Arrange them by increasing prediction score
    y_pred_knn_pred_sorted, idxs_knn_pred_sorted = tf.nn.top_k(tf.negative(y_pred_knn_dist_sorted), k=K)
    y_pred_knn_pred_sorted = tf.abs(y_pred_knn_pred_sorted)
    if debug: print('y_pred_knn_pred_sorted',y_pred_knn_pred_sorted.eval(),'idxs_knn_pred_sorted:',idxs_knn_pred_sorted.eval())
    # Then, argmin(idxs_knn_pred_sorted) (i.e. position of index 0) 
    # will correspond to position of index i in y_pred_knn_pred_sorted
    j = tf.argmin(idxs_knn_pred_sorted)
    if debug: print('j:',j.eval())
    
    # Get CDF of kNN(i): F_knn(i)_(s)
    cdf_y_pred_knn = tf.cumsum(y_pred_knn_pred_sorted)
    # Normalize
    cdf_y_pred_knn = tf.divide(cdf_y_pred_knn, cdf_y_pred_knn[-1])
    if debug: print('cdf_y_pred_knn',cdf_y_pred_knn.eval())
    # Get value at s=s_k, F_knn(i)_(s_k) using j from above:
    cdf_y_pred_knn_i = cdf_y_pred_knn[j]
    if debug: print('cdf_y_pred_knn_i:',cdf_y_pred_knn_i.eval())
    
    # Get CDF of corresponding uniform distn: F_(s)
    # Note: for uniform CDF, F_(s) = s (for increasing s)
    cdf_uniform = y_pred_knn_pred_sorted
    cdf_uniform_i = y_pred_i
    
    # Get grad_i: should be exact
    # NOTE: this expression from the paper seems to be dimensionally incorrect
    # If missing square, this expression would tend to dilute effect of deviant elements.
    grad_i = tf.multiply(2.,tf.subtract(cdf_y_pred_knn_i, cdf_uniform_i)) # TODO: determine correct sign and expression
    
    # Get loss_i: approximate since requires discrete integration over s = y_pred
    '''
    # Scheme 1:
    if debug: print(y_pred_knn_pred_sorted.eval())
    s_lo_edge = y_pred_knn_pred_sorted
    if debug: print('s_lo_edge',s_lo_edge.eval())
    s_hi_edge = tf.concat([tf.slice(y_pred_knn_pred_sorted, begin=[1], size=[K-1]),[1.]], axis=0)
    if debug: print('s_hi_edge',s_hi_edge.eval())
    ds = s_hi_edge - s_lo_edge
    if debug: print('ds',ds.eval())
    cdf_uniform_ = tf.reduce_mean(tf.stack([s_lo_edge, s_hi_edge]), axis=0)
    if debug: print(cdf_uniform_.eval())
    integrand = tf.pow(tf.subtract(cdf_y_pred_knn, cdf_uniform_),2.)
    if debug: print('integrand',integrand.eval())
    loss_i = tf.reduce_sum(tf.multiply(integrand, ds))
    if debug: print(loss_i.eval())
    '''
    # Scheme 2: 
    # Will have little effect if y_preds are uniformly dist'd but, in general,
    # will under-represent sparsely dist'd scores and over-represent densely dist'd scores
    integrand = tf.pow(tf.subtract(cdf_y_pred_knn, cdf_uniform),2.)
    if debug: print('integrand',integrand.eval())
    loss_i = tf.reduce_mean(integrand)
    if debug: print(loss_i.eval())
    
    return tf.stack([loss_i, grad_i])

# Debug with smaller K
cvm_i = cvm_loss_grad_i(m__[4], y_pred__, m__, 4, debug=True)
print('loss_i:',cvm_i[0].eval(),' grad_i:',cvm_i[1].eval())

m_i: [ 110.97079468]
dist [   6.66320038   17.14775848    4.89836121   18.22296143    0.
   27.47932434    6.67206573  267.69012451   74.61419678   21.17259216
    8.08588409   48.24597168   20.2829895   297.61682129   15.87670898
   93.71295166   95.46559143    4.89002228   12.07178497   39.24446106
   83.25579834   14.32927704    3.7778244    57.77528381   24.42472839
   15.23423767   11.66690063   13.19974518   29.72079468  111.90922546
   14.58078766  221.42474365]
idx: 4
y_pred_knn [ 0.73169005  0.25472414  0.97287232  0.51047093]
y_pred_knn_pred_sorted [ 0.25472414  0.51047093  0.73169005  0.97287232] idxs_knn_pred_sorted: [1 3 0 2]
j: 2
cdf_y_pred_knn [ 0.10313731  0.30982602  0.6060859   1.        ]
cdf_y_pred_knn_i: 0.606086
integrand [ 0.02297857  0.04025838  0.0157764   0.00073591]
0.0199373
loss_i: 0.0199373  grad_i: -0.251208


In [4]:
def cvm_loss_grad(m_, y_pred_, y_true, K, debug=False):
    '''
    Calculate Cramer-von Mises loss and gradient (with kNN modification) for this batch.
    See: https://arxiv.org/pdf/1410.4140.pdf
    '''
    
    # Only pick out bkg events (i.e. label=0)
    is_bkg = tf.equal(tf.cast(y_true, dtype=tf.int32), 0)
    y_pred = tf.boolean_mask(y_pred_, is_bkg)
    if debug: print('y_pred:',y_pred.eval())
    m = tf.boolean_mask(m_, is_bkg)
    if debug: print('m:',m.eval())
    
    cvm = tf.map_fn(lambda m_i: cvm_loss_grad_i(m_i, y_pred, m, K, debug=False), m, dtype=tf.float32)
    loss = tf.reduce_mean(cvm[:,0]) # assume each loss_i is equally weighted
    grad = cvm[:,1]
    
    #TODO: return grad for label=1 elements in y_pred as well (i.e. None)
    #grads = tf.zeros_like(y_pred_)
    #grads[is_bkg] = cvm[:,1]
    
    return loss, grad
    
# Debug
cvm = cvm_loss_grad(m__, y_pred__, y_true__, knn__, debug=True)
print('net loss:',cvm[0].eval())
print('grad wrt y_pred[is_bkg]:',cvm[1].eval())

y_pred: [[ 0.42541465]
 [ 0.50400704]
 [ 0.51047093]
 [ 0.01579145]
 [ 0.73169005]
 [ 0.99330503]
 [ 0.16287753]
 [ 0.12663478]
 [ 0.37483418]
 [ 0.69321942]
 [ 0.00290103]
 [ 0.36922905]
 [ 0.05867933]
 [ 0.78933609]
 [ 0.3497692 ]
 [ 0.70252371]
 [ 0.49122909]
 [ 0.97287232]
 [ 0.8359679 ]
 [ 0.61023736]
 [ 0.56451899]
 [ 0.99738914]
 [ 0.25472414]
 [ 0.01437786]
 [ 0.08904507]
 [ 0.93898261]
 [ 0.97345942]
 [ 0.49148154]
 [ 0.34093598]
 [ 0.72285759]
 [ 0.01089676]
 [ 0.7599532 ]]
m: [[ 104.3075943 ]
 [  93.82303619]
 [ 106.07243347]
 [ 129.1937561 ]
 [ 110.97079468]
 [ 138.45011902]
 [ 104.29872894]
 [ 378.66091919]
 [ 185.58499146]
 [  89.79820251]
 [ 119.05667877]
 [ 159.21676636]
 [  90.68780518]
 [ 408.58761597]
 [ 126.84750366]
 [ 204.68374634]
 [ 206.43638611]
 [ 115.86081696]
 [ 123.04257965]
 [ 150.21525574]
 [ 194.22659302]
 [  96.64151764]
 [ 107.19297028]
 [ 168.74607849]
 [ 135.39552307]
 [  95.73655701]
 [ 122.63769531]
 [  97.7710495 ]
 [ 140.69158936]
 [ 222.88002014